In [1]:
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib import RDF
import sys
sys.path.append('../../../')
sys.path.append('../../../andre')

import andre.utils as utils
from andre.utils import schema as SCHEMA
from andre.utils import pbs as PBS

# Récupération des documents XML


In [2]:
import xml.etree.ElementTree as ET
filePattern = "../ONIX_Files/202308211746_onix3_M3_{}.xml"

ns = { 'd': "http://ns.editeur.org/onix/3.0/reference" }


items = [] 
for n in range (0,347):
    file = filePattern.format(n)
    data = ET.parse(file)
    root = data.getroot()
    products = root.findall("./d:Product", ns)
    items += products
    

print(len(items), " livres au total")

29599  livres au total


# authors gathering (prequisite)

In [6]:
graphe_Auteurs = Graph()
graphe_Auteurs.bind("schema","http://schema.org/", override=True, replace=True)
graphe_Auteurs.bind("pbs","http://www.example.org/pbs#", override=True)
graphe_Auteurs.bind("author","http://schema.org/Person", override=True, replace=True)

id = 0
auteurs = {} # clé = Nom, Prénom
for item in items:
    editeur = item.findall(".//d:PublishingRole[.='01']../d:PublisherName", ns) 
    if not editeur:
        print('ERREUR',item)   

    liste_auteurs = item.findall('.//d:DescriptiveDetail/d:Contributor', ns)
    for a in liste_auteurs: 
        nom = a.find(".//d:PersonNameInverted", ns)
        if nom == None:
            continue
        if nom.text not in auteurs:
            id += 1
            auteurs[nom.text] = SCHEMA['author' + str(id)]
            graphe_Auteurs.add((SCHEMA['author' + str(id)],RDF.type, SCHEMA['Person']))
            nom_prenom = nom.text.rsplit(",")
            if len(nom_prenom) == 2:
                nom_famille = nom_prenom[0].strip()
                prenom = nom_prenom[1].strip()
                graphe_Auteurs.add((SCHEMA['author' + str(id)],SCHEMA.givenName,Literal(prenom)))
                graphe_Auteurs.add((SCHEMA['author' + str(id)],SCHEMA.familyName,Literal(nom_famille)))
            elif len(nom_prenom) == 1: # pseudonyme ex: Barasui ou nom/prenom pas clairement défini ex: Vieira da Silva
                nom_famille = nom_prenom[0].strip()
                graphe_Auteurs.add((SCHEMA['author' + str(id)],SCHEMA.familyName,Literal(nom_famille)))
                # print(nom.text)
            else:
                print(nom.text)

                
            bio = a.findall('./d:BiographicalNote', ns)
            if bio:
                graphe_Auteurs.add((SCHEMA['author' + str(id)],SCHEMA.description	,Literal(bio[0].text)))

            # print(nom.text, auteurs[nom])

# Publisher imprint graph creation (for test purposes)

In [3]:
graphe_Editeurs = Graph()
graphe_Editeurs.bind("schema","http://schema.org/", override=True, replace=True)

id = 0


editeurs = {}

for root in items:
    # editeur = root.findall(".//d:PublishingRole[.='01']../d:PublisherName", ns) 
    editeur = root.findall(".//d:ImprintIDType[.='01']../../d:ImprintName", ns)
    if not editeur:
        print('ERREUR',item)
    for e in editeur:
        editeurs[e.text] = ""
        
    
for n in editeurs:
    editeurs[n] = SCHEMA['publisher' + str(id)]
    graphe_Editeurs.add((SCHEMA['publisher' + str(id)], RDF.type, SCHEMA['publisher']))   
    graphe_Editeurs.add((SCHEMA['publisher' + str(id)], SCHEMA.name, Literal(n))) 
    id += 1
    
print(editeurs)

fichier = open("../Graphes/grapheEditeurs_Imprint_andre.ttl","wb")
graphe_Editeurs.serialize(fichier) 
fichier.close()

{'Gallimard': rdflib.term.URIRef('http://schema.org/publisher0'), 'les Trois ourses': rdflib.term.URIRef('http://schema.org/publisher1'), 'One Stroke': rdflib.term.URIRef('http://schema.org/publisher2'), 'Fleurus': rdflib.term.URIRef('http://schema.org/publisher3'), 'Autrement': rdflib.term.URIRef('http://schema.org/publisher4'), 'Flammarion': rdflib.term.URIRef('http://schema.org/publisher5'), 'Autrement Jeunesse': rdflib.term.URIRef('http://schema.org/publisher6'), 'le Navire en pleine ville': rdflib.term.URIRef('http://schema.org/publisher7'), "Canopé-CRDP de l'académie de Créteil": rdflib.term.URIRef('http://schema.org/publisher8'), 'BNF-Centre national de la littérature pour la jeunesse': rdflib.term.URIRef('http://schema.org/publisher9'), 'Mango-Jeunesse': rdflib.term.URIRef('http://schema.org/publisher10'), 'Quiquandquoi': rdflib.term.URIRef('http://schema.org/publisher11'), 'Magnard jeunesse': rdflib.term.URIRef('http://schema.org/publisher12'), 'Soleil': rdflib.term.URIRef('ht

In [4]:
# correspondance age/ niveau scolaires -> voir image
age_ranges = {
    "PC03" : [0, 1, 2, 3],
    "PC04" : [3, 4, 5, 6],
    "PC05" : [6, 8, 9],
    "PC06" : [9, 10, 11, 12],
    "PC07" : [12, 13, 14, 15],
    "PC01" : [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], # Tout Public: > 8 ans car sinon, longueur et format livre incompatible entre ages extrêmes
    "NS03" : [12, 13, 14, 15, 16],
    "NS02" : [6, 7, 8, 9, 10, 11],
    "NS01" : [0, 1, 2, 3, 4, 5], # prescolaire: < 6 ans
    "NS05" : [17, 18] # college = CEGEP
    }

In [7]:
graphe_Livres = Graph()
graphe_Livres.bind("schema","http://schema.org/", override=True, replace=True)
graphe_Livres.bind("pbs","http://www.example.org/pbs#", override=True)

livres = []
for root in items:
    rf = root.find('.//d:RecordReference', ns)
    graphe_Livres.add((SCHEMA[rf.text], RDF.type, SCHEMA['Book']))
    graphe_Livres.add((SCHEMA[rf.text], PBS.infoSource, PBS.BTLF))

    editeursLivre = root.findall(".//d:ImprintIDType[.='01']../../d:ImprintName", ns)

    for e in editeursLivre:
        graphe_Livres.add((SCHEMA[rf.text], SCHEMA.publisher, editeurs[e.text]))
   
        
    liste_auteurs = root.findall('.//d:Contributor', ns)
    if len(liste_auteurs) > 0:
        for a in liste_auteurs: 
            nom = a.find(".//d:ContributorRole[.='A01']../d:PersonNameInverted", ns)
            if nom != None:
                graphe_Livres.add((SCHEMA[rf.text], SCHEMA.author, auteurs[nom.text]))

        for a in liste_auteurs: 
            nom = a.find(".//d:ContributorRole[.='A12']../d:PersonNameInverted", ns)
            if nom != None:
                graphe_Livres.add((SCHEMA[rf.text], SCHEMA.illustrator, auteurs[nom.text]))

    isbn = root.findall(".//d:ProductIdentifier/d:ProductIDType[.='15']../d:IDValue", ns)
    if isbn:
        graphe_Livres.add((SCHEMA[rf.text], SCHEMA.isbn, Literal(isbn[0].text)))

    
       
    language = root.find('.//d:Language/d:LanguageCode', ns)
    if language != None:
        graphe_Livres.add((SCHEMA[rf.text], SCHEMA.inLanguage, Literal(language.text)))

    

    liste_thema_main_subjet = root.findall(".//d:Subject/d:MainSubject/../d:SubjectSchemeIdentifier[.='93']../d:SubjectCode", ns)
    for t in liste_thema_main_subjet:
        graphe_Livres.add((SCHEMA[rf.text], PBS.mainSubjectThema, Literal(t.text)))

                  
    liste_thema_subject = root.findall(".//d:Subject/d:SubjectSchemeIdentifier[.='93']../d:SubjectCode", ns)
    for t in liste_thema_subject:
        graphe_Livres.add((SCHEMA[rf.text], PBS.subjectThema, Literal(t.text)))
        
        
    # titre = root.findall('.//d:TitleStatement', ns)
    # titre = root.find(".//d:DescriptiveDetail/d:TitleDetail/d:TitleType[.='01']../d:TitleElement/d:TitleElementLevel[.='01']../d:TitleWithoutPrefix", ns)
    titre = root.find(".//d:DescriptiveDetail/d:TitleDetail/d:TitleType[.='01']../d:TitleElement/d:TitleElementLevel[.='01']../../d:TitleStatement", ns)

    if titre != None:
        graphe_Livres.add((SCHEMA[rf.text], SCHEMA.name, Literal(titre.text)))        
        
        
    description = root.findall(".//d:TextContent/d:TextType[.='03']../d:Text", ns)
    if description:
        graphe_Livres.add((SCHEMA[rf.text], SCHEMA.abstract, Literal(description[0].text)))
       
        
    reviewQuote = root.findall(".//d:TextContent/d:TextType[.='06']../d:Text", ns)
    if reviewQuote:
        graphe_Livres.add((SCHEMA[rf.text], SCHEMA.review, Literal(reviewQuote[0].text)))


    date = root.find(".//d:PublishingDate/d:Date", ns)
    if date != None:
        graphe_Livres.add((SCHEMA[rf.text], SCHEMA.datePublished, Literal(date.text)))
        
    audience = root.find(".//d:Audience/d:AudienceCodeValue", ns)
    if audience != None and audience.text in age_ranges:
        age_range = age_ranges[audience.text]
        for age in age_range:     
            graphe_Livres.add((SCHEMA[rf.text], PBS.age, Literal(age)))

fichier = open("../Graphes/grapheLivres_BTLF_Imprints.ttl","wb")
graphe_Livres.serialize(fichier) 
fichier.close()